In [68]:
import pandas as pd

gender_submission = pd.read_csv('./titanic_dataset/gender_submission.csv')
test_df = pd.read_csv('./titanic_dataset/test.csv')
train_df = pd.read_csv('./titanic_dataset/train.csv')

In [69]:
test_df = test_df.merge(gender_submission,on='PassengerId',how='inner')

In [73]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64
PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Ticket          0
Fare            1
Cabin           0
Embarked        0
dtype: int64


In [127]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


y_train = train_df['Survived']
X_train = train_df.drop(columns=['Survived', 'Name'])

y_test = test_df['Survived']
X_test = test_df.drop(columns=['Survived', 'Name'])

cat_features = ['Sex','Ticket','Cabin','Embarked']
X_train[cat_features] = X_train[cat_features].fillna('missing')
X_test[cat_features] = X_test[cat_features].fillna('missing')

j = np.arange(0.05,1.005,0.05)
j = [round(val, 2) for val in j]
array = []


model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.001,
    depth=3,
    loss_function='Logloss',
    cat_features=cat_features,
    verbose=50
    )

model.fit(X_train,
        y_train,
        eval_set=(X_test,y_test),
        early_stopping_rounds=50,
        use_best_model=True
        )

y_pred = model.predict(X_test)


0:	learn: 0.6927532	test: 0.6926242	best: 0.6926242 (0)	total: 8.67ms	remaining: 858ms


50:	learn: 0.6748252	test: 0.6672747	best: 0.6672747 (50)	total: 685ms	remaining: 658ms
99:	learn: 0.6586650	test: 0.6445433	best: 0.6445433 (99)	total: 1.33s	remaining: 0us

bestTest = 0.6445433073
bestIteration = 99



In [128]:
print('Acurácia:',accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

Acurácia: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           1       1.00      1.00      1.00       152

    accuracy                           1.00       418
   macro avg       1.00      1.00      1.00       418
weighted avg       1.00      1.00      1.00       418

